In [1]:
# pyscience imports
import os
import sys
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-darkgrid")
# plt.style.use("dark_background")
sns.set(style="ticks", context="talk")
# %matplotlib inline
# run for jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
from pprint import pprint
import re
import string
import numpy as np
import pandas as pd
import copy
import itertools
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from urllib import request
from collections import Counter
import json
from joblib import Parallel, delayed
import multiprocessing

import nltk
from nltk import word_tokenize
from nltk import bigrams
from nltk import trigrams
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk import word_tokenize

In [3]:
dbhome   = '/home/alal/Dropbox/0_GradSchool/1_HW/452/Hansard/'
ricehome = '/home/apoorval/HW/452/Hansard/'
home = ricehome
twfy_root = home + 'input/twfy/'
twfy = twfy_root + '2010+/'
jsons = home + 'input/twfy/jsons/'
%cd $home

/home/apoorval/HW/452/Hansard


# Parse XMLs 
Dry run

In [6]:
file = twfy+'debates2017-11-23c.xml'

In [7]:
tree = ET.parse(file)

In [19]:
for x in ET.iterparse(file):
    obj = x[1]
    tag = obj.tag
    
    text = obj.text
    if text:
        text = text.strip()

oral-heading
major-heading
p
speech
minor-heading
p
speech
p
speech
p
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
minor-heading
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
minor-heading
p
speech
p
speech
p
speech
p
speech
p
speech
phrase
p
speech
p
speech
p
speech
p
speech
phrase
p
speech
p
speech
minor-heading
p
speech
p
speech
p
speech
p
speech
phrase
phrase
p
speech
p
p
speech
p
speech
minor-heading
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
minor-heading
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
minor-heading
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
minor-heading
p
speech
phrase
p
p
speech
p
speech
p
speech
p
speech
p
speech
p
p
speech
p
speech
p
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
p
speech
major-heading
p

## Function to parse XML and populate list of speeches 

In [17]:
def twfy_parser(infile, inpdir = twfy, outdir = jsons):
    tree = ET.parse(inpdir + infile)
    root = tree.getroot()
    list_of_speeches = []
    for sp in tree.findall('speech'):
        # dict with metadata
        d =  sp.attrib
        # speech tag
        s = sp.findall('p')
        if s:
            # extract speeches
            statement = [x.text for x in s if x is not None]
            statement = ' '.join([x for x in statement if x])
        d['speech'] = statement
        list_of_speeches.append(d)
    # if there is a single speech
    if len(list_of_speeches) > 0:
        outfile = outdir + infile.replace('xml', 'json')
        json.dump(list_of_speeches, open(outfile, 'w'))
        return list_of_speeches

In [18]:
parl = os.listdir(twfy)

## Parallelise Parsing and Writing to JSON

In [19]:
%%time
num_cores = multiprocessing.cpu_count()
res = Parallel(n_jobs = num_cores)(delayed(twfy_parser)(p) for p in parl)

CPU times: user 4.48 s, sys: 1.34 s, total: 5.82 s
Wall time: 16.4 s


In [20]:
len(res)

1730

In [31]:
speeches = [x for x in res if x is not None]
len(speeches)
big_speech_list = [speech for day in speeches for speech in day]

1569

In [33]:
len(big_speech_list)

434719

In [35]:
outfile = jsons + '_all_speeches.json'
json.dump(big_speech_list, open(outfile, 'w'))

# Read JSONs

In [36]:
big_speech_list = json.load(open(jsons + '_all_speeches.json'))

In [38]:
len(big_speech_list)

434719

In [47]:
speakers = {}

In [48]:
big_speech_list[:5]

[{'id': 'uk.org.publicwhip/debate/2017-10-09a.74.1',
  'speakername': 'Chris Stephens',
  'person_id': 'uk.org.publicwhip/person/25306',
  'colnum': '74',
  'time': '18:19:00',
  'url': '',
  'speech': 'On a point of order, Mr Speaker. Last month, '},
 {'id': 'uk.org.publicwhip/debate/2011-10-17a.603.2',
  'nospeaker': 'true',
  'colnum': '603',
  'time': '14:30:00',
  'url': 'http://www.publications.parliament.uk/pa/cm201011/cmhansrd/cm111017/debtext/111017-0001.htm#111017-0001.htm_dpthd0',
  'speech': 'The Secretary of State was asked—'},
 {'id': 'uk.org.publicwhip/debate/2011-10-17a.603.4',
  'speakerid': 'uk.org.publicwhip/member/40395',
  'speakername': 'Annette Brooke',
  'oral-qnum': '1',
  'colnum': '603',
  'time': '14:30:00',
  'url': 'http://www.publications.parliament.uk/pa/cm201011/cmhansrd/cm111017/debtext/111017-0001.htm#1110177000414',
  'speech': 'What assessment he has made of the 2011 GCSE results for academies; and if he will make a statement.'},
 {'id': 'uk.org.pub

In [49]:
for x in big_speech_list:
    speaker = x.get("speakerid", None)
    if speaker is not None:
        speakers[speaker] = x['speakername']

In [55]:
speaks = pd.DataFrame.from_dict(speakers, orient = 'index')
speaks.columns = ['person_name']

In [56]:
speaks.head()

,person_name
uk.org.publicwhip/member/40395,Annette Brooke
uk.org.publicwhip/member/40097,Nick Gibb
uk.org.publicwhip/member/40419,Chi Onwurah
uk.org.publicwhip/member/40455,James Gray
uk.org.publicwhip/member/40373,Stephen Twigg


In [61]:
%cd $twfy_root
%mkdir tmp -p

/home/apoorval/HW/452/Hansard/input/twfy


In [62]:
speaks.to_csv(twfy_root + 'tmp/speaker_list.csv')